In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
import food.custom_pandas as cpd
from tqdm import tqdm
import requests 

# !nbdev_build_lib

search best match of food classes to images (5)

- request glovo api for best match  
- create sql table corresponding photo_clips
- save clips to sql table

In [5]:
query = f"""select f.id, f.text, f.description from food.foods_prompted f
            left join food.foods_prompted_images i on (f.id = i.food_id)
            where i.clip is null"""

bs = 1


total = engine.execute(f'select count(*) from ({query}) a').one()

pd_iter = cpd.read_sql_query(query, engine, chunksize=1, index_col='id')

for df in tqdm(pd_iter, desc=f"clip image inference {schema}", total=total[0] // bs):  
    food_id = df.iloc[0].name
    text = df.iloc[0]['text']
    description = df.iloc[0]['description']

    url = f"http://localhost:8184/search"
    r_df = pd.read_json(requests.post(url,params={'text':text,'topk':5,'return_clip':True}).json())[['country_code','store_name','product_name','path','accuracy','clip']]
    r_df['food_id'] = food_id
    r_df.to_sql('foods_prompted_images', engine, if_exists='append',schema=schema,index=False)



clip image inference food: 0it [00:00, ?it/s]
